### Используемые библиотеки

In [2]:
import pandas as pd
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from skopt.space import Categorical, Real
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# считываем обработанные данные
df = pd.read_csv("data.csv")
df_y = df[['IC50, mM', 'CC50, mM', 'SI']].copy()
df_x = df.drop(columns=['IC50, mM', 'CC50, mM', 'SI'])

# добавляем новый признак согласно заданию
median = df_y['CC50, mM'].median()
df_y['bool'] = (df_y['CC50, mM'] > median).astype(bool)


### Классификация CC50 (превышение медианного значения выборки)

In [4]:
# датасет для сравнения метрик по каждой из модели
metrics = pd.DataFrame(columns=["Accuracy",
                                "Precision",
                                "Recall",
                                "F1-score",
                                "ROC AUC"]
                                )

In [5]:
# избавимся от лишних признаков, выделим целевую переменную и добавим дополнительный признак
y_data = df_y['bool']
x_data = df_x
x_data['IC50, mM'] = df_y['IC50, mM']

# сократим количество признаков до 50 с помощью Mutual Info
selector = SelectKBest(score_func=mutual_info_regression, k=50)
select_x_data = selector.fit_transform(x_data, y_data)
selected_features = x_data.columns[selector.get_support()]
x_data = pd.DataFrame(select_x_data, columns=selected_features)

# разделим данные на тестовыые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(f'Тренеровочные данные: {X_train.shape}, {y_train.shape}')
print(f'Tестовые данные: {X_test.shape}, {y_test.shape}')

# стандартизируем данные train
scaler = StandardScaler()
tmp_data = scaler.fit_transform(X_train)
scaled_X_train = pd.DataFrame(tmp_data, columns=x_data.columns)

# стандартизируем данные test
tmp_data = scaler.transform(X_test)
scaled_X_test = pd.DataFrame(tmp_data, columns=x_data.columns)

Тренеровочные данные: (772, 50), (772,)
Tестовые данные: (194, 50), (194,)


#### RandomForestClassifier

In [6]:
# определим пространство гиперпараметров для модели RandomForestClassifier
param_space_rfr = {
    'n_estimators': (50, 500),
    'max_depth': (1, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# создадим модель
model = RandomForestClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_rfr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
rfc_model = bayes_search.best_estimator_


ROC AUC = 0.8767

Лучшие параметры модели:
bootstrap = True
max_depth = 15
max_features = sqrt
min_samples_leaf = 2
min_samples_split = 2
n_estimators = 263


In [7]:
# расcчитаем y
y_pred = rfc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[0] = res

Accuracy: 0.8557
Precision: 0.8488
Recall: 0.8295
F1-score: 0.8391


#### GradientBoostingClassifier

In [8]:
# определим пространство гиперпараметров для модели GradientBoostingClassifier
param_space_gbr = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 7),
}

# создадим модель
model = GradientBoostingClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_gbr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
gbc_model = bayes_search.best_estimator_

ROC AUC = 0.8919

Лучшие параметры модели:
max_depth = 19
min_samples_leaf = 5
min_samples_split = 9
n_estimators = 89


In [9]:
# расcчитаем y
y_pred = gbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[1] = res

Accuracy: 0.8505
Precision: 0.8734
Recall: 0.7841
F1-score: 0.8263


#### CatBoostClassifier

In [10]:
# определим пространство гиперпараметров для модели CatBoostClassifier
param_space_cbc = {
    'iterations': (100, 1000),
    'depth': (4, 10),
    'grow_policy': Categorical(['SymmetricTree', 'Depthwise', 'Lossguide']),
    'loss_function': Categorical(['Logloss', 'CrossEntropy']),
    'l2_leaf_reg': Real(1, 10),
    'min_data_in_leaf': (1, 50),
    'one_hot_max_size': (2, 10)
}

# создадим модель
model = CatBoostClassifier()

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_cbc,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
cbc_model = bayes_search.best_estimator_

0:	learn: 0.6731447	total: 63.8ms	remaining: 60s
0:	learn: 0.6735223	total: 65.6ms	remaining: 1m 1s
0:	learn: 0.6752117	total: 66.4ms	remaining: 1m 2s
1:	learn: 0.6582652	total: 71.9ms	remaining: 33.7s
1:	learn: 0.6559134	total: 72.4ms	remaining: 34s
1:	learn: 0.6575973	total: 77.9ms	remaining: 36.6s
2:	learn: 0.6381419	total: 81.5ms	remaining: 25.4s
2:	learn: 0.6400362	total: 82ms	remaining: 25.6s
2:	learn: 0.6423703	total: 87.9ms	remaining: 27.5s
3:	learn: 0.6269970	total: 93.4ms	remaining: 21.8s
0:	learn: 0.6752909	total: 67.7ms	remaining: 1m 3s
3:	learn: 0.6223692	total: 98.8ms	remaining: 23.1s
0:	learn: 0.6732538	total: 68.4ms	remaining: 1m 4s
3:	learn: 0.6259380	total: 105ms	remaining: 24.5s
1:	learn: 0.6576910	total: 79ms	remaining: 37s
4:	learn: 0.6138272	total: 110ms	remaining: 20.6s
4:	learn: 0.6076249	total: 114ms	remaining: 21.3s
1:	learn: 0.6554720	total: 81.9ms	remaining: 38.4s
4:	learn: 0.6098125	total: 119ms	remaining: 22.3s
2:	learn: 0.6390384	total: 95.9ms	remaining: 

In [11]:
# расcчитаем y
y_pred = cbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[2] = res

Accuracy: 0.866
Precision: 0.878
Recall: 0.8182
F1-score: 0.8471


In [12]:
metrics.index = ['RandomForestClassifier', 'GradientBoostingClassifier', 'CatBoostClassifier']
metrics.head()

,Accuracy,Precision,Recall,F1-score,ROC AUC
RandomForestClassifier,0.8557,0.8488,0.8295,0.8391,0.8767
GradientBoostingClassifier,0.8505,0.8734,0.7841,0.8263,0.8919
CatBoostClassifier,0.8660,0.8780,0.8182,0.8471,0.8910


#### Вывод
Было выполнено преобразование признака СC50, mM по бинарному критерию. Критерий оценки состоял из сравнения медианного значения признака СC50, mM с текущим значением данных.

Таким образом данные превосзодившие медианное значение СC50, mM получали значение True, равные или меньшие получали значение False.

Для булевой целевой переменной СC50 были дообработанны данные:
* Добавлен дополнительный признак IC50, mM в данные для обучения
* Сокращено количество признаков до 50
* Данные разделены на тестовыые и тренировочные
* Проведена стандартизация тренировочных данных и на её основе стандартизированы тестовые данные

Затем были подобраны параметры и построены три модели:

* RandomForestClassifier c параметрами:
    - bootstrap = True
    - max_depth = 15
    - max_features = sqrt
    - min_samples_leaf = 2
    - min_samples_split = 2
    - n_estimators = 263

* GradientBoostingClassifier с параметрами:
    - max_depth = 19
    - min_samples_leaf = 5
    - min_samples_split = 9
    - n_estimators = 89

* CatBoostClassifier с параметрами:
    - depth = 9
    - grow_policy = SymmetricTree
    - iterations = 638
    - l2_leaf_reg = 8.224883030406795
    - loss_function = CrossEntropy
    - min_data_in_leaf = 6
    - one_hot_max_size = 8

В качестве метрик для оценки работы модели использовались:
* Accuracy
* Precision
* Recall
* F1-score
* ROC AUC

Наилучшие показатели при обучение получились у модели CatBoostClassifier, однако после валидации на тестовых данных, модель GradientBoostingClassifier показала наилучшие результаты.